In [31]:
import polars as pl

In [32]:
# Import incidents and format columns
df = pl.read_csv(
    "./data/incident_dump.csv",
).with_columns(
    pl.col("reported")
    .str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%S%z")
    .dt.convert_time_zone("America/Chicago"),
    pl.col("reported_date").str.to_date("%Y-%m-%d"),
    pl.col("validated_location").str.split(",").cast(pl.List(pl.Float64)),
    pl.col("incident").str.to_lowercase(),
)

In [33]:
df.groupby(["incident"]).agg(pl.count()).sort("count", descending=True)

incident,count
str,u32
"""theft""",2956
"""information""",1366
"""found property…",914
"""lost property""",828
"""traffic violat…",533
"""information / …",488
"""liquor law vio…",440
"""medical call""",420
"""battery""",418


In [34]:
excluded_list = [
    "Fondling",
    "Medical Call",
    "Luring a Minor",
    "Lost Property",
    "Stalking",
    "Sexual Assault",
    "Dating",
    "Stalking",
    "Domestic",
    "Sex",
    "Found Property",
    "Mental Health",
    "Harassment by Electronic Means",
    "Well-Being",
    "Threatening Phone Call",
    "Medical Transport",
    "Warrant",
    "Lost Wallet",
    "Fire Alarm",
    "Chemical Spill",
    "Suspicious Mail",
    "Eavesdropping",
    "Sex Offense",
    "Sex Offender",
    "Sex Crime",
    "Domestic Aggravated Battery",
    "Dating Violence",
    "Harassing Messages",
]
df = df.filter(~pl.col("incident").str.contains("|".join(excluded_list)))
df.groupby(["incident"]).agg(pl.count()).sort("count", descending=True)

incident,count
str,u32
"""theft""",2956
"""information""",1366
"""found property…",914
"""lost property""",828
"""traffic violat…",533
"""information / …",488
"""liquor law vio…",440
"""medical call""",420
"""battery""",418


In [35]:
def list_to_parsed_set(unparsed_list: [str]):
    unparsed_list.sort()
    parsed_set = set()
    for element in unparsed_list:
        if "/" in element:
            for p in element.split(" / "):
                fmt_element = p.strip().lower()
                if p:
                    parsed_set.add(fmt_element)
        else:
            fmt_element = element.strip().lower()
            parsed_set.add(fmt_element)
    return parsed_set

In [36]:
print(list_to_parsed_set(df["incident"].to_list()))

{'damage to property', 'obstructing a peace officer', 'theft', 'dating violence', 'theft from motor vehicles', 'fleeing or attempting to elude police', 'non-criminal damage', 'theft of motorcycle', 'cyberstalking', 'suspicious person', 'theft and recovery of motor vehicle', 'non criminal incident', 'other dangerous weapon', 'attempted vehicular hijacking', 'hate crime', 'possession of a controlled substance', 'obscene phone calls', 'mislaid property', 'damage to city property', 'order of protection arrest', 'traffic arrest', 'threatening phone call', 'resisting arrest', 'harassment by electronic means', 'possession of stolen motor vehicle', 'indecent exposure', 'found', 'possession of cannabis', 'possession of drug paraphernalia', 'traffic violation', 'vandalism', 'other vehicle offenses', 'bomb threat', 'illness', 'deceptive practice', 'harassing phone call`', 'attempt theft from motor vehicle', 'assault and harassment by electronic means', 'unlawful use of weapons', 'aggravated vehic